In [1]:
import numpy as np
import pandas as pd
import os
from scipy.io import loadmat
import scipy.io as sio
from sklearn.utils import shuffle 
import matplotlib.pyplot as plt

# Labels

In [2]:
file_path = 'E:/STUDY/Publications/Thesis/Brain Emotion Detection/Dataset/Practice with datasets/Seed/label.mat'
mat_data = sio.loadmat(file_path)
#Explore Variable Names

variable_names = mat_data.keys()
labels = mat_data["label"].flatten()

print(labels)


[ 1  0 -1 -1  0  1 -1  0  1  1  0 -1  0  1 -1]


# Feature Extract of each trial and saving in .npy format

EEG Bands\
0: delta: 1-3Hz\
1: theta: 4-7Hz\
2: alpha: 8-13Hz\
3: beta: 14-30Hz \
4: gamma: 31-50Hz

In [3]:
def feature_extractor_to_npy(mat_folder, save_folder, Feature_type, labels):
    trial_num = 1
    
    # Loop through all files in the folder
    for filename in os.listdir(mat_folder):
        if filename.endswith(".mat"):
            mat_file = os.path.join(mat_folder, filename)
            loaded_data = loadmat(mat_file)

            # Extract human number from the filename
            human_number = int(filename.split('_')[0])

            # Loop through the variables in the loaded file
            for i in range(1, 16):  # Assuming you want to loop from 1 to 15
                var_name = f"{Feature_type}{i}"
                if var_name in loaded_data and isinstance(loaded_data[var_name], np.ndarray) and len(loaded_data[var_name].shape) == 3:
                    trial_data = loaded_data[var_name]
                    label = labels[i - 1]  # Adjust index since it starts from 1
                    np.save(os.path.join(save_folder, f"subject_{human_number}_{trial_num}_{Feature_type}_label_({label}).npy"), trial_data)
                    trial_num += 1

    print("Saving complete.")


In [4]:
# Example usage:
mat_folder = "E:/STUDY/Publications/Thesis/Brain Emotion Detection/Dataset/SEED/SEED_EEG/ExtractedFeatures"
save_folder = "E:/STUDY/Publications/Coonference 1/Dataset/SEED_PSD_LDS_X_y/All Trials"


type = "psd_LDS"
feature_extractor_to_npy(mat_folder, save_folder, type, labels)

Saving complete.


# Ensemble in X & Y

In [5]:
def ensemble_and_save(directory_path, output_path):
    # Initialize arrays to store data sizes and maximum size
    sizes = []
    max_vals = 0

    # Loop through all files in the folder
    for filename in os.listdir(directory_path):
        if filename.endswith(".npy"):
            npy_file = os.path.join(directory_path, filename)
            loaded_data = np.load(npy_file)
            # Loop through the variables in the loaded file
            sizes.append(loaded_data.shape[1])
            max_vals = max(max_vals, loaded_data.shape[1])

    #print("Maximum size:", max_vals)

    # Initialize arrays for features and labels
    features = np.zeros((675, 62, max_vals, 5))
    label_list = np.zeros(675)
    trial = 0

    # Loop through all files in the data folder
    for filename in os.listdir(directory_path):
        if filename.endswith(".npy"):
            # Extract information from the filename
            parts = filename.split('_')
            subject_number = int(parts[1])
            trial_number = int(parts[2])
            feature_type = parts[3]
            # Extracting the label and removing parentheses
            label_str = parts[-1][:-4]  # Removing ".npy" extension
            label_str = label_str.replace('(', '').replace(')', '')  # Removing parentheses
            label = int(label_str)

            # Load the feature data
            feature_data = np.load(os.path.join(directory_path, filename), allow_pickle=True)

            # Add data and labels to the lists
            features[trial, :62, :feature_data.shape[1], :] = feature_data
            label_list[trial] = label

            trial += 1

    # Convert lists to NumPy arrays
    X = np.array(features)
    y = np.array(label_list)

    # Save X and y to the specified output path
    np.save(os.path.join(output_path, 'X.npy'), X)
    np.save(os.path.join(output_path, 'y.npy'), y)

    print("Saved successfully!")




In [6]:
# Example usage
data_directory = "E:/STUDY/Publications/Coonference 1/Dataset/SEED_PSD_LDS_X_y/All Trials"
output_directory = "E:/STUDY/Publications/Coonference 1/Dataset/SEED_PSD_LDS_X_y"
ensemble_and_save(data_directory, output_directory)

Saved successfully!
